In [1]:
import pandas as pd
import numpy as np

# 1035, 258
train = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/spmm-latency-traintest/train-test-csv/nonsquare-train-1035-from-spmm-contain-todense-over-3s-1293.csv')
test = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/spmm-latency-traintest/train-test-csv/nonsquare-test-258-from-spmm-contain-todense-over-3s-1293.csv')

In [2]:
# Train + Valid
X_train = train[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd','lnnz*rnnz']] 
y_train = train['sp_smdm']

# Test
X_test = test[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd','lnnz*rnnz']] 
y_test = test['sp_smdm']

In [3]:
# 데이터 표준화(Standardization)
# 회귀 문제에선 MinMaxScaler가 좋음
from sklearn.preprocessing import MinMaxScaler

# 변형 객체 생성
minmax_scaler = MinMaxScaler()

# 훈련데이터의 모수 분포 저장
minmax_scaler.fit(X_train)

# 훈련 데이터 스케일링
X_train = minmax_scaler.transform(X_train)

# 테스트 데이터의 스케일링
X_test = minmax_scaler.transform(X_test)

In [4]:
from sklearn.metrics import mean_squared_error

def mean_absolute_percentage_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

def rmse(y_true, y_pred):
    rmse = np.sqrt(np.mean(np.square(y_pred - y_true))) 
    return rmse

def custom_scoring(y_test, y_pred):

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)

    return mape

In [5]:
# from sklearn.neural_network import MLPRegressor
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import KFold
# from sklearn.metrics import make_scorer

# grid = {
#     'hidden_layer_sizes' : [],
#     'learning_rate_init' : [0.0005,0.001,0.005]
# }

# regressor = MLPRegressor(max_iter=5000,early_stopping=True)

# regressor_grid = GridSearchCV(regressor, 
#                               param_grid = grid, 
#                               scoring = make_scorer(custom_scoring, greater_is_better=False),
#                               n_jobs=-1,
#                               verbose=3) 

# regressor_grid.fit(X_train, y_train)

# print("최고의 파라미터 :", regressor_grid.best_params_)
# print("최고 평균 정확도 : {}".format(regressor_grid.best_score_)) 

In [6]:
# from sklearn.neural_network import MLPRegressor
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import KFold
# from sklearn.metrics import make_scorer

# kf = KFold(random_state=30,
#            n_splits=10,
#            shuffle=True,
#           )

# grid = {
#     'hidden_layer_sizes' : [(16,),(32,),(64,)],
#     'alpha' : [0.0001,0.0005,0.001, 0.005, 0.01],
#     'learning_rate_init' : [0.005, 0.01, 0.05, 0.1],
# }

# regressor = MLPRegressor(max_iter=5000)

# regressor_grid = GridSearchCV(regressor, 
#                               param_grid = grid, 
#                               scoring = make_scorer(custom_scoring, greater_is_better=False),
#                               cv=kf, 
#                               n_jobs=-1,
#                               verbose=3) 

# regressor_grid.fit(X_train, y_train)

# print("최고의 파라미터 :", regressor_grid.best_params_)
# print("최고 평균 정확도 : {}".format(regressor_grid.best_score_))

In [9]:
from sklearn.neural_network import MLPRegressor

regressor = MLPRegressor(hidden_layer_sizes=(1024, 256, 64, 16, 1,), 
                         max_iter=1000,
                         activation='relu',
                         alpha=0.01,
                         learning_rate_init=0.001,
                         early_stopping=False
                         )

regressor.fit(X_train, y_train)

y_pred_train = regressor.predict(X_train)
y_pred = regressor.predict(X_test)

In [10]:
# 훈련 자체 에러율
print("--------train 에러율--------")
print("rmse : {}".format(rmse(y_train,y_pred_train)))
print("mape : {}".format(mean_absolute_percentage_error(y_train, y_pred_train)))

# 테스트 에러율
print("--------test 에러율--------")
print("rmse : {}".format(rmse(y_test,y_pred)))
print("mape : {}".format(mean_absolute_percentage_error(y_test,y_pred)))

--------train 에러율--------
rmse : 22955.068915097167
mape : 19.38236518374932
--------test 에러율--------
rmse : 27559.838830254357
mape : 22.10957880464378
